In [1]:
from background_code.host_password import *      # 导入密码和host
from background_code.mysqlconnection import mysql
import pandas as pd
import time

In [2]:
# 计算交易日----------------------------------------------------------------------------

# 连接交易日数据库
data_test_1 = mysql('tradedays')
sql = 'select * from `hs_tradedays_daily`'
tradedays_daily = pd.DataFrame(list(data_test_1.execute(sql)))
tradedays_daily

,0
0,20100104
1,20100105
2,20100106
3,20100107
4,20100108
...,...
2669,20201225
2670,20201228
2671,20201229
2672,20201230


In [3]:
# An algorithm in marking first & last trade day in a week with mistakes
def mark_first_end_value(week_list,index):
    
    # 一旦一个日期被确定为某周第一个交易日，周号将被储存;一旦周号被存储，则周号对应的一定不是第一个交易日
    save_value=0         
    
    lis=[]
    
    if index==0:    # 当index=0时，mark每周第一个交易日
        # 方法就是把周号颠倒过来,lis在颠倒回去
        for weekofyear in week_list:
            # 判断周号是不是某周第一个交易日，即：确定flag的值
            if save_value!=weekofyear:    # 周号未被储存或更新时
                lis.append(1)             # 该周号是某周第一个交易日，记做1
                save_value=weekofyear     # 该周号被储存后，就不再是第一个交易日了
            else:                         # 周号已被被储存或更新时
                lis.append(0)             # 该周号不是某周第一个交易日，记做0       

        return lis
    
    else:         # 当index=1时，mark每周最后一个交易日
        # 方法就是把周号颠倒过来,lis在颠倒回去
        for weekofyear in week_list[::-1]:
            # 判断周号是不是某周第一个交易日，即：确定flag的值
            if save_value!=weekofyear:    # 周号未被储存或更新时
                lis.append(1)             # 该周号是某周第一个交易日，记做1
                save_value=weekofyear
            else:                         # 周号已被被储存或更新时
                lis.append(0)             # 该周号不是某周第一个交易日，记做0

        return lis[::-1]

In [4]:
# Correct wrong mark near New Year
def corr_error(tradedays_daily):
    
    # 推论：函数mark_first_end_value只可能在跨年时出现问题
    error_occur=tradedays_daily[((tradedays_daily['endday']==1) | 
                                 (tradedays_daily['startday']==1))
                               &
                               ((tradedays_daily['weekofyear']==0) | 
                                (tradedays_daily['weekofyear']==52) |
                               (tradedays_daily['weekofyear']==53))]
     
# 命题：跨年问题出现的充要条件：前一年的最后一个交易日和新一年的第一个交易日出现在一个自然周里
# 命题：跨年问题出现的充要条件是：前一年的最后一个交易日出现在最后一周，
#                            新一年的第一个交易日出现在第一周，
#                            新年第一个交易日的周几数大于前一年的最后一个交易日的周几数,
#                            跨年前后两个交易日日期差小于等于5天
#                            

    for i in error_occur.index:
        # 如果一年的第一个交易日出现在weekofyear=1时(即：一年的第二周)，那就不存在跨年问题
        # 推论：当且仅当，一年的的第一周(weekyear=0)出现交易日时，才可能出现跨年问题
        # 推论：i-1行和i行的天数之差大于5，则不会发生跨年问题;
        #      该推论等价于：当且仅当上一年最后一周(weekyear=0)出现交易日时，才可能出现跨年问题
        #      由于有的时候是52周有的时候是53周
        # 推论：若i行为0周，i-1行不为0周，问题只可能出现在i-1行的endday和i行的startday
        if (tradedays_daily.at[i,'weekofyear']==0 and tradedays_daily.at[i-1,'weekofyear']!=0 
            and tradedays_daily.at[i,'dayofweek']>tradedays_daily.at[i-1,'dayofweek']
            and (pd.to_datetime(tradedays_daily.at[i,0],format='%Y%m%d')-
            pd.to_datetime(tradedays_daily.at[i-1,0],format='%Y%m%d')).days<=5):
            # 如果i行是0周，i-1行不是0周，那i-1行一定是上一年的52或53周的最后一个交易日，
            # i-1行的startday一定是对的，i-1行endday有可能是错的
            # i行的endday一定是对的，i行的startday有可能是错的
            # 推论：当且仅当，i行的周几数大于i-1行的周几数时，会发生跨年问题
            tradedays_daily.at[i,'startday']=0
            tradedays_daily.at[i-1,'endday']=0

    return tradedays_daily

In [5]:
# 寻找每周第一个和最后一个交易日
def find_fisrt_end_day(tradedays_daily,ref_date,indicator):

    ref_date_index = list(tradedays_daily[0]).index(int(ref_date))
    
    if indicator==0:             # 寻找从自己开始，离自己最近的startday
        for i in range(ref_date_index,-1,-1):        # 一组倒序数列，从自己开始，左闭右开
            if tradedays_daily.at[i,'startday']==1:
                break
    elif indicator==1:           # 寻找从自己的上一个开始，离自己最近的,上一个endday
        for i in range(ref_date_index-1,-1,-1):     # 一组倒序数列，从自己的上一行开始，左闭右开
            if tradedays_daily.at[i,'endday']==1:
                break
    elif indicator==-1:          # 寻找从自己开始，离自己最近的，下一个endday
        # 一组正数列，从自己开始，前往最后一行，左闭右开
        for i in range(ref_date_index,len(tradedays_daily)):
            if tradedays_daily.at[i,'endday']==1:
                break

    return str(tradedays_daily.at[i,0])

In [6]:
tradedays_daily['dayofweek']=tradedays_daily[0].apply(
    lambda x: pd.to_datetime(x,format='%Y%m%d').dayofweek+1)
tradedays_daily['weekofyear']=tradedays_daily[0].apply(
    lambda x: int(pd.to_datetime(x,format='%Y%m%d').strftime("%W")))
tradedays_daily['startday']=mark_first_end_value(list(tradedays_daily['weekofyear']),0) 
tradedays_daily['endday']=mark_first_end_value(list(tradedays_daily['weekofyear']),1)
tradedays_daily=corr_error(tradedays_daily)
tradedays_daily

,0,dayofweek,weekofyear,startday,endday
0,20100104,1,1,1,0
1,20100105,2,1,0,0
2,20100106,3,1,0,0
3,20100107,4,1,0,0
4,20100108,5,1,0,1
...,...,...,...,...,...
2669,20201225,5,51,0,1
2670,20201228,1,52,1,0
2671,20201229,2,52,0,0
2672,20201230,3,52,0,0


In [7]:
# ref_date could be any trade day
ref_date='20201203'
ref_date_start=find_fisrt_end_day(tradedays_daily,ref_date,0);print(ref_date_start)
last_ref_date=find_fisrt_end_day(tradedays_daily,ref_date,1);print(last_ref_date)
last_ref_start=find_fisrt_end_day(tradedays_daily,last_ref_date,0);print(last_ref_start)
ll_ref=find_fisrt_end_day(tradedays_daily,last_ref_date,1);print(ll_ref)

20201130
20201127
20201123
20201120


In [8]:
# 当n=1时，返回上一个ref_date；当n=k时，返回上k个ref_date
# 在m个ref_date中，当n=k时，返回第(m-k)个ref_date
def ref_recursive(n):    
    if n==1:
        return find_fisrt_end_day(tradedays_daily,ref_date,1)
    else:
        return find_fisrt_end_day(tradedays_daily,ref_recursive(n-1),1)

In [9]:
ref_recursive(4)

'20201106'